# EuroPython 2018

Martin Christen, July 2018<br/>
martin.christen@fhnw.ch

**Processing Geodata using Python and Open Source Modules**

In this notebook the following will be covered:

Vector File Formats (Shapefiles, GeoJSON, KML, GeoPackage)
Raster File Formats (GeoTiff, ...)

* Shapely: Manipulation and analysis of geometric objects ( https://github.com/Toblerity/Shapely )
* Fiona - The pythonic way to handle vector data ( https://github.com/Toblerity/Fiona )
* rasterio - The pythonic way to handle raster data ( https://github.com/mapbox/rasterio )
* pyproj - transforming spatial reference systems ( https://github.com/jswhit/pyproj ) 
* Geospatial analysis with GeoPandas ( https://github.com/geopandas/geopandas ) 
* Creating maps using Folium ( https://github.com/python-visualization/folium )

In this notebook the following data is used:

* Natural Earth Dataset, https://www.naturalearthdata.com/ (public domain)
* Blue Marble: Next Generation was produced by Reto Stöckli, NASA Earth Observatory (NASA Goddard Space Flight Center)
* geonames: http://www.geonames.org/ CC License

## 1. Installation

(This tutorial requires anaconda, if you don't have it yet, download it here: https://www.anaconda.com/download/ )

**This notebook requires Python 3.6 (or higher)** (jupyter notebook not lab)

### 1.1 Installing Modules

Install the main modules using conda, dependencies will be resolved (gdal etc.)

    conda install shapely
    conda install fiona
    conda install rasterio
    conda install geopandas
    conda install folium -c conda-forge

## 2. Shapely

Shapely is a BSD-licensed Python package for **manipulation** and **analysis** of **planar geometric objects**. 

* Shapely is **not** concerned with data formats or coordinate systems.
* Shapely is based on the widely deployed GEOS (the engine of PostGIS) and JTS (from which GEOS is ported) libraries.

### 2.1 Simple Feature Access

**Simple Feature Access** is both an Open Geospatial Consortium (OGC) and International Organization for Standardization (ISO) standard **ISO 19125** that specifies a common storage and access model of mostly two-dimensional geometries (point, line, polygon, multi-point, multi-line, etc.) used by geographic information systems.

Shapely supports the following Features:

* Point
* LineString
* LinearRing          
* Polygon
* MultiLineString
* MultiPoint
* MultiPolygon


### 2.2 Spatial Data Model

The fundamental types of geometric objects implemented by Shapely are points, curves, and surfaces. Each is associated with three sets of (possibly infinite) points in the plane. The interior, boundary, and exterior sets of a feature are mutually exclusive and their union coincides with the entire plane

* A Point has an interior set of exactly one point, a boundary set of exactly no points, and an exterior set of all other points. A Point has a topological dimension of 0.

* A Curve has an interior set consisting of the infinitely many points along its length (imagine a Point dragged in space), a boundary set consisting of its two end points, and an exterior set of all other points. A Curve has a topological dimension of 1.

* A Surface has an interior set consisting of the infinitely many points within (imagine a Curve dragged in space to cover an area), a boundary set consisting of one or more Curves, and an exterior set of all other points including those within holes that might exist in the surface. A Surface has a topological dimension of 2.

### 2.3 Examples

In [ ]:
from shapely.geometry import Polygon

polygon1 = Polygon([(30, 10), (40, 40), (20, 35), (10, 20), (30, 10)])

print(f"Polygon area: {polygon1.area}, polygon length: {polygon1.length}") 

You can display shapely objects directly in the Jupyter Notebook. It is more or less a "debug" output.

It is also possible to display this in matplotlib/descartes, we will see later how to bring this directly on a map)

In [ ]:
polygon1

In [ ]:
polygon2 = Polygon([(20,20),(80,30),(50,40),(20,20)])
polygon2

In [ ]:
polygon2.union(polygon1)

In [ ]:
polygon2.intersection(polygon1)

In [ ]:
polygon2.symmetric_difference(polygon1)

In [ ]:
result = polygon2.symmetric_difference(polygon1)

print(f"Polygon area: {result.area}, polygon length: {result.length}")

In [ ]:
result.wkt

In [ ]:
s = result.wkt
type(s)

In [ ]:
import shapely.wkt

mypolygon = shapely.wkt.loads(s)
mypolygon

There are also several binary operations available:

- **contains** (Returns True if the interior of the object intersects the interior of the other but does not contain it, and the dimension of the intersection is less than the dimension of the one or the other.)
- **intersects** (Returns True if the boundary and interior of the object intersect in any way with those of the other.)
- **witin** (Returns True if the object’s boundary and interior intersect only with the interior of the other (not its boundary or exterior).
- **touches** (Returns True if the objects have at least one point in common and their interiors do not intersect with any part of the other.)
- **crosses** (Returns True if the interior of the object intersects the interior of the other but does not contain it, and the dimension of the intersection is less than the dimension of the one or the other.)
- **equals** (Returns True if the set-theoretic boundary, interior, and exterior of the object coincide with those of the other.)

In [ ]:
polygon1.intersects(polygon2)

In [ ]:
polygon1.within(polygon2)

In [ ]:
polygon1.equals(polygon1)

## 3. Fiona

Spatial information is not only geometry. Together with it we always have metadata (properties). Lets look at a first dataset.


### 3.1 Reading Vector Data

In [ ]:
import fiona

c = fiona.open('data/ne_10m_airports/ne_10m_airports.shp', 'r')

airport = next(iter(c))
airport

In [ ]:
airport['properties']['name']

In [ ]:
airport['geometry']['type']

In [ ]:
airport['geometry']['coordinates']

In [ ]:
c.close()

Lets read all data. There are basically two ways:

a) load everything to memory: (if dataset isn't too large...)

    alldata = list(c)
    
b) iterate through all data: (one by one):

    for element in c:
        ...
    

In [ ]:
with fiona.open('data/ne_10m_airports/ne_10m_airports.shp', 'r') as c:
    for airport in c:
        if airport['properties']['iata_code'] == "EDI":
            print(airport['properties']['name'])
            print(airport['geometry']['coordinates'])
            print(airport['properties']['wikipedia'])        


Coordinate System ? (Coordinate Reference System?)

In [ ]:
with fiona.open('data/ne_10m_airports/ne_10m_airports.shp', 'r') as c:
    print(c.crs)

EPSG: 4326 look it up at: http://epsg.io/

( http://epsg.io/4326 )


For transformations to other spatial reference systems, use **pyproj**


Lets look at another dataset:

In [ ]:
import fiona

c = fiona.open('data/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp', 'r')

country = next(iter(c))
country

In [ ]:
print(country['properties']['NAME'])

In [ ]:
print(country['properties']['NAME_ZH'])

In [ ]:
print(country['properties']['CONTINENT'])

In [ ]:
print(country['properties']['POP_EST'])
print(country['properties']['POP_YEAR'])

In [ ]:
with fiona.open('data/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp', 'r') as c:
    for country in c:
        if country['properties']['NAME'] == "United Kingdom":
            print(country['properties']['POP_EST'])
            print(country['properties']['POP_YEAR'])   
            print(country['geometry']['type'])
            # print(country['geometry']['coordinates']) # you don't want to print all coordinates!
            geometry = country['geometry']


### 3.2 Example: find Airports in UK

In [ ]:
from shapely.geometry import shape
from shapely.geometry import MultiPolygon

uk = MultiPolygon(shape(geometry))
uk

In [ ]:
from shapely.geometry import Point

with fiona.open('data/ne_10m_airports/ne_10m_airports.shp', 'r') as c:
    for airport in c:      
        position = Point(airport['geometry']['coordinates'])
        if position.within(uk):
            print(airport['properties']['iata_code'], airport['properties']['name'], airport['geometry']['coordinates'])

## 4. Rasterio

### 4.1 Reading data

    rasterio.open(fp, mode='r', driver=None, width=None, height=None, count=None, crs=None, transform=None, dtype=None, nodata=None, **kwargs)
    
    
#### Parameters:	
**fp**: string or file
  A filename or URL, or file object opened in binary mode.

**mode**: string
“r” (read), “r+” (read/write), or “w” (write)

**driver**: string
Driver code specifying the format name (e.g. “GTiff” or “JPEG”). See GDAL docs at http://www.gdal.org/formats_list.html (optional, required for writing).

**width**: int
Number of pixels per line (optional, required for write).

**height**: int
Number of lines (optional, required for write).

**count**: int > 0
Count of bands (optional, required for write).

**dtype**: rasterio.dtype
the data type for bands such as rasterio.ubyte for 8-bit bands or rasterio.uint16 for 16-bit bands (optional, required for write)

**crs**: dict or string
Coordinate reference system (optional, recommended for write).

**transform**: Affine instance
Affine transformation mapping the pixel space to geographic space (optional, recommended for writing).

**nodata**: number
Defines pixel value to be interpreted as null/nodata (optional, recommended for write, will be broadcast to all bands).

Open gibt entweder ein "**DatasetReader**" Objekt zurück (beim lesen) oder ein "**DatasetUpdater**" Objekt.


In [ ]:
import rasterio

dataset = rasterio.open('data/BlueMarble.tif', 'r')

In [ ]:
dataset.name

In [ ]:
dataset.mode

In [ ]:
dataset.count # number of raster bands, in our case 3 for r,g,b

In [ ]:
dataset.indexes

In [ ]:
dataset.width

In [ ]:
dataset.height

In [ ]:
dataset.crs

In [ ]:
dataset.affine  # affine transformation pixel to crs

In [ ]:
dataset.affine * (0, 0)

In [ ]:
~dataset.affine # inverse affine transformation crs to pixel

In [ ]:
~dataset.affine * (0,0)

In [ ]:
px,py = ~dataset.affine * (-3.209626, 55.946167) # Edinburgh (EICC) to pixel coordinates! 
print(px,py)

In [ ]:
dataset.bounds

In [ ]:
print(dataset.bounds.left)
print(dataset.bounds.bottom)
print(dataset.bounds.right)
print(dataset.bounds.top)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
r = dataset.read(1)
g = dataset.read(2)
b = dataset.read(3)

In [ ]:
rgb = np.dstack((r,g,b))  # stack r,g,b so we can display it...
rgb

In [ ]:
fig, ax = plt.subplots(figsize=(15,9))
ax.imshow(rgb, interpolation='nearest')
ax.plot(px,py, 'ro'); # Edinburgh EICC

## 5. Geopandas

GeoPandas is an open source project to make working with geospatial data in python easier. GeoPandas **extends** the datatypes used by pandas to **allow spatial operations** on **geometric types**. Geometric operations are performed by shapely. Geopandas further depends on fiona for file access and descartes and matplotlib for plotting.
(geopandas.org)


In [ ]:
import pandas as pd

df = pd.read_csv('data/cities5k.csv', encoding="utf-8", sep=",", header=None, low_memory=False)
df.head()

In [ ]:
df2 = df[[1,4,5,14]]
df2.columns = ["name", "lat", "lng", "population"]
df2.head()

In [ ]:
df2.query("name == 'Edinburgh'")

Now lets create a geopandas dataframe. Basically just create a new column "geometry" - using Shapely to create Points

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

geometry = [Point(pos) for pos in zip(df2['lng'], df2['lat'])]
gdf = gpd.GeoDataFrame(df2, geometry=geometry)

In [ ]:
gdf.head()

In [ ]:
gdf = gdf.drop(['lat', 'lng'], axis=1) # remove redundant data

In [ ]:
gdf.plot()

In [ ]:
# Export to Shapefile

gdf.to_file("cities.shp", driver="Shapefile", encoding="utf-8")



In [ ]:
cities = gdf
cities.head()  # store "cities for later"

In [ ]:
# another query:

bigcities = cities[cities.population > 10000000]

bigcities = bigcities.sort_values(['population'], ascending=False)
bigcities

Now we load the admin 0 country files again:

In [ ]:
countries = gpd.read_file("data/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp", encoding="utf-8")
countries.head()

In [ ]:
countries2 = countries[["NAME", "POP_EST", "geometry"]]
countries2.head()

In [ ]:
uk = countries2[countries2['NAME'] == "United Kingdom"]
uk

In [ ]:
uk.plot(figsize=(15,9))

In [ ]:
restoftheworld = countries2[countries2['NAME'] != "United Kingdom"]

ax = uk.plot(figsize=(15,9), color="red")
restoftheworld.plot(ax=ax, color="blue");

In [ ]:
#type(uk.geometry) # GeoSeries
type(uk.iloc[0].geometry) # shapely.geometry.multipolygon.MultiPolygon

In [ ]:
# Get All Cities within UK

ukcities = cities[cities.within(uk.iloc[0].geometry)]
ukcities.head()

## 6. Folium

In [ ]:
import folium
map_osm = folium.Map(location=[55.946167, -3.209626], zoom_start=17)
map_osm

### GeoPandas & Folium

In [ ]:
# our previous geodataframe
uk

In [ ]:
import folium

center = [55.946167, -3.209626] # EICC
map_uk = folium.Map(center, zoom_start=3)   


folium.GeoJson(uk).add_to(map_uk)

map_uk

In [ ]:
import folium

center = [55.946167, -3.209626] # EICC
map_uk = folium.Map(center, zoom_start=3)   

folium.GeoJson(uk,style_function=lambda feature: {
        'fillColor': 'green',   # you can also replace this with functions with feature as argument
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }).add_to(map_uk)

map_uk

In [ ]:
# our last map: display bigcities on a map using markers

bigcities

In [ ]:
world_map = folium.Map(location=[0,0], zoom_start=2)

def f(row):
    lng, lat = row[2].x, row[2].y
    name = row[0]
    population = row[1]
    folium.Marker([lat, lng], 
              popup=name + "<br/>" + str(int(population)),
              icon=folium.Icon(color="red", prefix="fa", icon="home")).add_to(world_map)

bigcities.apply(f, axis=1);
    
world_map



In [ ]:
world_map.save("mymap.html")